In [15]:
import pandas as pd
import numpy as np
import seaborn as sns

In [16]:
df=pd.read_csv("/Users/mayurimamdi/Downloads/netflix_titles.csv")

In [17]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [18]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [19]:
df[df['rating'].isnull()].dropna(axis=0)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


In [20]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [21]:
df.dropna(subset=['rating'], inplace=True)

In [22]:
df.isna().sum()

show_id            0
type               0
title              0
director        2631
cast             825
country          830
date_added        10
release_year       0
rating             0
duration           3
listed_in          0
description        0
dtype: int64

In [23]:
df.duplicated().sum()

np.int64(0)

First we will do movie reccomendation system

In [24]:
movie_df=df[df['type']=='Movie']

In [25]:
movie_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...


In [31]:
movie_df= movie_df[['title','rating','listed_in','description']]

In [99]:
movie_df['combined_text'] = (
    movie_df['title'] + " " +
    movie_df['listed_in'] + " " +
    movie_df['description']
)


In [100]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

le = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def data_processing(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    words = text.split()

    clean_words = [
        le.lemmatize(word)
        for word in words
        if word not in stop_words
    ]

    return " ".join(clean_words)

movie_df['clean_text'] = movie_df['combined_text'].apply(data_processing)

print("Text cleaning done")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayurimamdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mayurimamdi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mayurimamdi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Text cleaning done


In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2)
)

tfidf_matrix = tfidf.fit_transform(movie_df['clean_text'])

print("TF-IDF Shape:", tfidf_matrix.shape)


TF-IDF Shape: (6129, 10000)


In [102]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)


In [103]:
indices = pd.Series(
    movie_df.index,
    index=movie_df['title']
).drop_duplicates()


In [104]:
def recommend_movie(title, cosine_sim=cosine_sim):

    if title not in indices:
        return "Movie not found"

    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(
        sim_scores,
        key=lambda x: x[1],
        reverse=True
    )

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return movie_df[['title','listed_in']].iloc[movie_indices]


In [105]:
def evaluate_genre_overlap(title):

    if title not in indices:
        print("Movie not found.")
        return

    idx = indices[title]
    original_genre = movie_df.loc[idx, 'listed_in']

    original_set = set(original_genre.split(', '))

    recommendations = recommend_movie(title)

    overlap_count = 0

    for movie in recommendations['title']:

        if movie not in indices:
            continue

        rec_idx = indices[movie]
        rec_genre = movie_df.loc[rec_idx, 'listed_in']

        rec_set = set(rec_genre.split(', '))

        if original_set.intersection(rec_set):
            overlap_count += 1

    print(f"Original Genre: {original_genre}")
    print(f"Genre Match Count: {overlap_count} / {len(recommendations)}")


In [106]:
recommend_movie("Sankofa")


,title,listed_in
1515,Guest House,Comedies
8211,The Black Room,Horror Movies
1117,Ibrahim a Fate to Define,"Documentaries, International Movies"
4502,Mr. Chandramouli,"Dramas, International Movies, Thrillers"
2513,Light in the Dark,"Dramas, International Movies"


In [108]:
evaluate_genre_overlap("Sankofa")

Original Genre: Dramas, Independent Movies, International Movies
Genre Match Count: 3 / 5


In [109]:
recommend_movie("The Starling")

,title,listed_in
2579,Go! Go! Cory Carson: The Chrissy,Children & Family Movies
1892,A Go! Go! Cory Carson Halloween,Children & Family Movies
1635,A Go! Go! Cory Carson Christmas,Children & Family Movies
2166,A Go! Go! Cory Carson Summer Camp,Children & Family Movies
151,Initial D,"Action & Adventure, International Movies"


In [110]:
evaluate_genre_overlap("The Starling")


Original Genre: Comedies, Dramas
Genre Match Count: 0 / 5


In [111]:
evaluate_genre_overlap("Je Suis Karl")

Original Genre: Dramas, International Movies
Genre Match Count: 2 / 5


In [112]:
evaluate_genre_overlap("Minsara Kanavu")

Original Genre: Comedies, International Movies, Music & Musicals
Genre Match Count: 5 / 5
